# First step in pipeline
**Load HDBSCAN and t-SNE models and find probabilities for all sentences in a document. The output of this file will be used in the segmentation**

In [1]:
import os
import pickle
import hdbscan
import numpy as np
import pandas as pd
from openTSNE import TSNE
from bertopic import BERTopic
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer

In [6]:
# load PCA class object
with open('pca_model_50.pkl', 'rb') as inp:
    PCA_model = pickle.load(inp)

In [3]:
# load t-SNE class object
with open('sentence_tsne_data_.pkl', 'rb') as inp:
    tsne_data = pickle.load(inp)

In [4]:
# load cluster model
with open('sentence_hdbscan_model.pkl', 'rb') as inp:
    hdbscan_model = pickle.load(inp)

In [7]:
# loop through all of the annotated transcripts and save their topic probability distribution
directory = '../Thesis/annotated_transcripts_input'

for dirpath, _, files in os.walk(directory):
    for file in files:
        episode_id = file.split('_')[0]
        path = os.path.join(dirpath, file)
        with open(path, errors='replace') as f: 
            prediction_documents = pd.read_csv(f)
            docs = prediction_documents.transcript_subset

            # embedd
            sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
            embedded_sentences = sentence_model.encode(docs)

            # reduce dimensions with PCA and t-SNE
            PCA_data = PCA_model.transform(embedded_sentences)
            print(f"{1-sum(PCA_model.explained_variance_ratio_):.2%} of the variance has been removed by PCA for transcript {episode_id}")
            tsne_prediction = tsne_data.transform(PCA_data)

            # get probability vectors for each cluster via soft clustering
            probabilities = hdbscan.membership_vector(hdbscan_model, tsne_prediction)
            print(f'nrows: {len(probabilities)}\nncols: {len(probabilities[0])}')
            # remedy HDBSCAN problem
            inds = np.where(np.isnan(probabilities))
            probabilities[inds] = 0
            print(f"Number of nan rows: {len(set(list(inds[:][0])))} for {episode_id}\n\n")

            # save probabilities to use in downstream segmentation
            name_of_file = f'../Thesis/annotated_probabilities/topic_probability_density_vector_{episode_id}.npy'
            np.save(name_of_file, probabilities)

51.06% of the variance has been removed by PCA for transcript 7A7swZJL0AtFghauiGLadV
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 4.36 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.04 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 1294.9848, 50 iterations in 31.2757 sec
Iteration  100, KL divergence 1265.0461, 50 iterations in 31.1954 sec
Iteration  150, KL divergence 1250.5674, 50 iterations in 33.2270 sec
Iteration  200, KL divergence 1244.0025, 50 iterations in 32.7587 sec
Iteration  250, KL divergence 1243.6578, 50 iterations in 40.4830 sec
   --> Time elapsed: 168.94 seconds
nrows: 98
ncols: 470
Number of nan rows: 0 for 7A7swZJL0AtFghauiGLadV


51.06% of the variance has been removed by PCA for transcript 5Sg6efUjy

/Users/oskarmunckafrosenschold/Documents/Thesis/thesisenv/lib/python3.10/site-packages/hdbscan/prediction.py:581: RuntimeWarning: All-NaN slice encountered
  outlier_vec = outlier_membership_vector(


nrows: 223
ncols: 470
Number of nan rows: 6 for 3DR5Qa40Mc17AiBYfmC29U


51.06% of the variance has been removed by PCA for transcript 0bXWB28GwN8OiqC1ykRrRX
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 2.46 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.01 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 8453.1513, 50 iterations in 36.0985 sec
Iteration  100, KL divergence 8281.5766, 50 iterations in 31.4605 sec
Iteration  150, KL divergence 8233.4261, 50 iterations in 32.9959 sec
Iteration  200, KL divergence 8212.3571, 50 iterations in 31.2729 sec
Iteration  250, KL divergence 8196.5726, 50 iterations in 30.1004 sec
   --> Time elapsed: 161.93 seconds


/Users/oskarmunckafrosenschold/Documents/Thesis/thesisenv/lib/python3.10/site-packages/hdbscan/prediction.py:581: RuntimeWarning: All-NaN slice encountered
  outlier_vec = outlier_membership_vector(


nrows: 563
ncols: 470
Number of nan rows: 4 for 0bXWB28GwN8OiqC1ykRrRX


51.06% of the variance has been removed by PCA for transcript 13NDTKL5ZGs8cb8dojW3bz
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 0.67 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.01 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 3107.0315, 50 iterations in 30.3416 sec
Iteration  100, KL divergence 3048.2820, 50 iterations in 30.1698 sec
Iteration  150, KL divergence 3032.4029, 50 iterations in 29.5258 sec
Iteration  200, KL divergence 3027.5012, 50 iterations in 29.4216 sec
Iteration  250, KL divergence 3026.7413, 50 iterations in 29.5344 sec
   --> Time elapsed: 148.99 seconds


...

nrows: 601
ncols: 470
Number of nan rows: 22 for 0ZGQ63222rqX5TD5ZrMmcN


51.06% of the variance has been removed by PCA for transcript 7mv5E2yb2yVQU34OiQ1vqv
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 1.18 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.01 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 11041.5129, 50 iterations in 33.9136 sec
Iteration  100, KL divergence 10846.0460, 50 iterations in 33.9683 sec
Iteration  150, KL divergence 10773.4136, 50 iterations in 33.3661 sec
Iteration  200, KL divergence 10747.1482, 50 iterations in 31.9839 sec
Iteration  250, KL divergence 10737.6245, 50 iterations in 32.7934 sec
   --> Time elapsed: 166.03 seconds


/Users/oskarmunckafrosenschold/Documents/Thesis/thesisenv/lib/python3.10/site-packages/hdbscan/prediction.py:581: RuntimeWarning: All-NaN slice encountered
  outlier_vec = outlier_membership_vector(


nrows: 747
ncols: 470
Number of nan rows: 32 for 7mv5E2yb2yVQU34OiQ1vqv


51.06% of the variance has been removed by PCA for transcript 3p9FLEH5V5sCGHhGubaYZc
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 0.98 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.01 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 2612.3163, 50 iterations in 32.2327 sec
Iteration  100, KL divergence 2565.5355, 50 iterations in 31.6417 sec
Iteration  150, KL divergence 2548.1779, 50 iterations in 31.7197 sec
Iteration  200, KL divergence 2546.6050, 50 iterations in 31.9051 sec
Iteration  250, KL divergence 2544.1366, 50 iterations in 31.6155 sec
   --> Time elapsed: 159.12 seconds
nrows: 187
ncols: 470
Number of nan rows: 0 for 3p9FLEH5V5sCGHhGubaYZ

/Users/oskarmunckafrosenschold/Documents/Thesis/thesisenv/lib/python3.10/site-packages/hdbscan/prediction.py:581: RuntimeWarning: All-NaN slice encountered
  outlier_vec = outlier_membership_vector(


nrows: 560
ncols: 470
Number of nan rows: 6 for 1VBbCB6ja5pPdU2wrBy27N




### Code for individual transcript predicitons

In [2]:
# load dataframe of one transcript with sentences as instances
sentences = pd.read_csv('first_podcast.csv.gz', compression='gzip')
documents = sentences.transcript_subset.to_list()

In [7]:
# embedd
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embedded_sentences = sentence_model.encode(documents)

In [8]:
# reduce dimensions with PCA and t-SNE
PCA_model = PCA(n_components = 50)
PCA_data = PCA_model.fit_transform(embedded_sentences)
print(f"{1-sum(PCA_model.explained_variance_ratio_):.2%} of the variance has been removed by PCA")

tsne_test = tsne_data.transform(PCA_data)

38.83% of the variance has been removed by PCA
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 11.78 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.03 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 9527.4070, 50 iterations in 70.7268 sec
Iteration  100, KL divergence 9287.1780, 50 iterations in 92.7301 sec
Iteration  150, KL divergence 9191.3463, 50 iterations in 104.2941 sec
Iteration  200, KL divergence 9151.7852, 50 iterations in 87.6444 sec
Iteration  250, KL divergence 9132.1676, 50 iterations in 86.1700 sec
   --> Time elapsed: 441.57 seconds


In [9]:
# get probability vectors for each cluster via soft clustering
probabilities = hdbscan.membership_vector(hdbscan_model, tsne_test)
print(f'nrows: {len(probabilities)}')
print(f'ncols: {len(probabilities[0])}')
# remedy for HDBSCAN problem
inds = np.where(np.isnan(probabilities))
probabilities[inds] = 0
print(f"Number of nan rows: {len(set(list(inds[:][0])))}")

/Users/oskarmunckafrosenschold/Documents/Thesis/thesisenv/lib/python3.10/site-packages/hdbscan/prediction.py:581: RuntimeWarning: All-NaN slice encountered
  outlier_vec = outlier_membership_vector(


nrows: 599
ncols: 156


In [11]:
# save probabilities to use in segmentation
np.save('probabilities.npy', probabilities)